In [1]:
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

In [6]:
admissions = pd.read_csv('data-ingest/data/admissions_hb_long_unfiltered.csv', parse_dates = [0])
admissions.columns = all_data.columns.str.lower()
admissions.head()

,date,health_board,admission_type,count
0,2020-04-01,Betsi Cadwaladr University Health Board,C19,28
1,2020-04-01,Betsi Cadwaladr University Health Board,NonC19,98
2,2020-04-01,Betsi Cadwaladr University Health Board,All,126
3,2020-04-01,Hywel Dda University Health Board,C19,16
4,2020-04-01,Hywel Dda University Health Board,NonC19,63


In [19]:
lockdowns = pd.read_csv('data-ingest/data/lockdown_status_hb_date.csv')
lockdowns['date'] = pd.to_datetime(lockdowns['date'], format = '%Y-%m-%d')
lockdowns.head()

,date,health_board,prop_lockdown,lockdown_status,proportion_lockdown
0,2020-04-01,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0
1,2020-04-02,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0
2,2020-04-03,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0
3,2020-04-04,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0
4,2020-04-05,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0


In [25]:
lockdowns.loc[lockdowns['health_board']=='Powys Teaching Health Board', 'lockdown_status'] = "No local lockdowns"
lockdowns[lockdowns['health_board']=='Powys Teaching Health Board']

,date,health_board,prop_lockdown,lockdown_status,proportion_lockdown
1398,2020-04-23,Powys Teaching Health Board,2.0,No local lockdowns,2.0
1399,2020-04-24,Powys Teaching Health Board,2.0,No local lockdowns,2.0
1400,2020-04-25,Powys Teaching Health Board,2.0,No local lockdowns,2.0
1401,2020-04-26,Powys Teaching Health Board,2.0,No local lockdowns,2.0
1402,2020-04-27,Powys Teaching Health Board,2.0,No local lockdowns,2.0
...,...,...,...,...,...
1604,2020-11-15,Powys Teaching Health Board,2.0,No local lockdowns,0.0
1605,2020-11-16,Powys Teaching Health Board,2.0,No local lockdowns,0.0
1606,2020-11-17,Powys Teaching Health Board,2.0,No local lockdowns,0.0
1607,2020-11-18,Powys Teaching Health Board,2.0,No local lockdowns,0.0


In [27]:
local_locks_start = dt.datetime.strptime('2020-09-08', '%Y-%m-%d')
local_locks_end = dt.datetime.strptime('2020-10-23', '%Y-%m-%d')

lockdowns = lockdowns[(lockdowns['date'] >= local_locks_start)&(lockdowns['date'] < local_locks_end)]
lockdowns.head()

,date,health_board,prop_lockdown,lockdown_status,proportion_lockdown
160,2020-09-08,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0
161,2020-09-09,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0
162,2020-09-10,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0
163,2020-09-11,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0
164,2020-09-12,Abertawe Bro Morgannwg University Health Board,0.0,No local lockdowns,0.0


In [32]:
lockdowns = lockdowns[lockdowns['health_board']!='Velindre University NHS Trust']
lockdowns.loc[lockdowns['health_board']=='Wales', 'lockdown_status'] = 'Mixed'
lockdowns = lockdowns.drop(columns = ['prop_lockdown', 'proportion_lockdown'])
lockdowns.tail()

,date,health_board,lockdown_status
2042,2020-10-18,Wales,Mixed
2043,2020-10-19,Wales,Mixed
2044,2020-10-20,Wales,Mixed
2045,2020-10-21,Wales,Mixed
2046,2020-10-22,Wales,Mixed


In [33]:
loc_adm = admissions.merge(lockdowns, how = 'inner', on = ['date', 'health_board'])
loc_adm.health_board.unique()

array(['Betsi Cadwaladr University Health Board',
       'Hywel Dda University Health Board',
       'Abertawe Bro Morgannwg University Health Board',
       'Cardiff and Vale University Health Board',
       'Cwm Taf University Health Board',
       'Aneurin Bevan University Health Board',
       'Powys Teaching Health Board', 'Wales'], dtype=object)

In [35]:
loc_adm = loc_adm[loc_adm['admission_type']=='C19'].drop(columns = ['health_board', 'admission_type'])
loc_adm

,date,count,lockdown_status
0,2020-09-08,15,No local lockdowns
3,2020-09-08,0,No local lockdowns
6,2020-09-08,5,No local lockdowns
9,2020-09-08,17,No local lockdowns
12,2020-09-08,12,No local lockdowns
...,...,...,...
1065,2020-10-22,32,Local lockdowns
1068,2020-10-22,25,Local lockdowns
1071,2020-10-22,18,Local lockdowns
1074,2020-10-22,0,No local lockdowns


In [41]:
loc_adm_grouped = loc_adm.groupby(['date', 'lockdown_status']).sum().reset_index()
loc_adm_grouped['ravg_admissions'] = loc_adm_grouped.groupby('lockdown_status')['count'].transform(lambda x: x.rolling(7, 1).mean())

loc_adm_grouped

,date,lockdown_status,count,ravg_admissions
0,2020-09-08,Local lockdowns,16,16.000000
1,2020-09-08,Mixed,65,65.000000
2,2020-09-08,No local lockdowns,49,49.000000
3,2020-09-09,Local lockdowns,10,13.000000
4,2020-09-09,Mixed,56,60.500000
...,...,...,...,...
130,2020-10-21,Mixed,92,90.857143
131,2020-10-21,No local lockdowns,0,0.000000
132,2020-10-22,Local lockdowns,107,91.571429
133,2020-10-22,Mixed,107,91.571429


In [46]:
fig = px.line(loc_adm_grouped, x = 'date', y = 'ravg_admissions', 
              color = 'lockdown_status', color_discrete_sequence = ['#ED8103', '#747875', '#50869D'])
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.show()